In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [2]:
class Greens:
    def read_les(self, datadir, filename, GorSigma):
        h5file = h5py.File(datadir + filename, 'r')
        self.nt = h5file['solve/params/nt'][()] + 1
        self.nao = h5file['solve/params/nao'][()]
        nt = self.nt
        nao = self.nao
        nao2 = nao**2
        self.GL = np.zeros((nt * nao, nt * nao), dtype = complex)
        GLlinear = h5file[GorSigma + '/GL']
        pos = 0
        for tp in np.arange(nt):
            for t in np.arange(tp+1):
                self.GL[tp*nao : (tp+1)*nao, t*nao : (t+1)*nao] = GLlinear[pos : pos + nao2].reshape((nao,nao))
                pos += nao2

    def read_ret(self, datadir, filename, GorSigma):
        h5file = h5py.File(datadir + filename, 'r')
        self.nt = h5file['solve/params/nt'][()] + 1
        self.nao = h5file['solve/params/nao'][()]
        nt = self.nt
        nao = self.nao
        nao2 = nao**2
        self.GR = np.zeros((nt * nao, nt * nao), dtype = complex)
        GRlinear = h5file[GorSigma + '/GR']
        pos = 0
        for t in np.arange(nt):
            for tp in np.arange(t+1):
                self.GR[t*nao : (t+1)*nao, tp*nao : (tp+1)*nao] = GRlinear[pos : pos+nao2].reshape((nao,nao))
                pos += nao2
        
    def read_tv(self, datadir, filename, GorSigma):
        h5file = h5py.File(datadir + filename, 'r')
        self.nt = h5file['solve/params/nt'][()] + 1
        self.ntau = h5file['solve/params/ntau'][()] + 1
        self.nao = h5file['solve/params/nao'][()]
        nt = self.nt
        ntau = self.ntau
        nao = self.nao
        nao2 = nao**2
        self.GTV = np.zeros((nt * nao, ntau * nao), dtype = complex)
        GTVlinear = h5file[GorSigma + '/GTV']
        pos = 0
        for t in np.arange(nt):
            for tau in np.arange(ntau):
                self.GTV[t*nao : (t+1)*nao, tau*nao : (tau+1)*nao] = GTVlinear[pos : pos+nao2].reshape((nao,nao))
                pos += nao2

    def __init__(self, datadir, filename, GorSigma):
        if ".h5" in filename:
            self.read_les(datadir, filename, GorSigma)
            self.read_ret(datadir, filename, GorSigma)
            self.read_tv(datadir, filename, GorSigma)
        if ".npy" in filename:
            self.GR = np.load(datadir + GorSigma + "R_block_" + filename)
            self.GL = np.load(datadir + GorSigma + "L_block_" + filename)
            self.GTV = np.load(datadir + GorSigma + "TV_block_" + filename)

            
    def npy_output(self, datadir, filename, GorSigma):
        np.save(datadir + GorSigma + "R_block_" + filename, self.GR)
        np.save(datadir + GorSigma + "L_block_" + filename, self.GL)
        np.save(datadir + GorSigma + "TV_block_" + filename, self.GTV)


class Greens_sep:
    def read_les(self, datadir, filename, GorSigma):
        h5file = h5py.File(datadir + filename, 'r')
        self.nt = h5file['solve/params/nt'][()] + 1
        self.nao = h5file['solve/params/nao'][()]
        nt = self.nt
        nao = self.nao
        nao2 = nao**2
        self.GL = np.zeros((nt, nt, nao, nao), dtype = complex)
        GLlinear = h5file[GorSigma + '/GL']
        pos = 0
        for tp in np.arange(nt):
            for t in np.arange(tp+1):
                self.GL[tp, t] = GLlinear[pos:pos+nao2].reshape((nao,nao))
                pos += nao2
        self.GL = np.swapaxes(self.GL, 1,3)
        self.GL = np.swapaxes(self.GL, 0,2)


    def read_ret(self, datadir, filename, GorSigma):
        h5file = h5py.File(datadir + filename, 'r')
        self.nt = h5file['solve/params/nt'][()] + 1
        self.nao = h5file['solve/params/nao'][()]
        nt = self.nt
        nao = self.nao
        nao2 = nao**2
        self.GR = np.zeros((nt, nt, nao, nao), dtype = complex)
        GRlinear = h5file[GorSigma + '/GR']
        pos = 0
        for t in np.arange(nt):
            for tp in np.arange(t+1):
                self.GR[t, tp] = GRlinear[pos:pos+nao2].reshape((nao,nao))
                pos += nao2
        self.GR = np.swapaxes(self.GR, 1,3)
        self.GR = np.swapaxes(self.GR, 0,2)
        
    def read_tv(self, datadir, filename, GorSigma):
        h5file = h5py.File(datadir + filename, 'r')
        self.nt = h5file['solve/params/nt'][()] + 1
        self.ntau = h5file['solve/params/ntau'][()] + 1
        self.nao = h5file['solve/params/nao'][()]
        nt = self.nt
        ntau = self.ntau
        nao = self.nao
        nao2 = nao**2
        self.GTV = np.zeros((nt, ntau, nao, nao), dtype = complex)
        GTVlinear = h5file[GorSigma + '/GTV']
        pos = 0
        for t in np.arange(nt):
            for tau in np.arange(ntau):
                self.GTV[t, tau] = GTVlinear[pos:pos+nao2].reshape((nao,nao))
                pos += nao2
        self.GTV = np.swapaxes(self.GTV, 1,3)
        self.GTV = np.swapaxes(self.GTV, 0,2)


    def __init__(self, datadir, filename, GorSigma):
        if ".h5" in filename:
            self.read_les(datadir, filename, GorSigma)
            self.read_ret(datadir, filename, GorSigma)
            self.read_tv(datadir, filename, GorSigma)
        if ".npy" in filename:
            self.GR = np.load(datadir + GorSigma + "R_sep_" + filename)
            self.GL = np.load(datadir + GorSigma + "L_sep_" + filename)
            self.GTV = np.load(datadir + GorSigma + "TV_sep_" + filename)

            
    def npy_output(self, datadir, filename, GorSigma):
        np.save(datadir + GorSigma + "R_sep_" + filename, self.GR)
        np.save(datadir + GorSigma + "L_sep_" + filename, self.GL)
        np.save(datadir + GorSigma + "TV_sep_" + filename, self.GTV)


In [40]:
datadir = '/pauli-storage/tblommel/NEdyson_data/He-VB2PP/equilibrium/'
filename = 'nt1000_k5_dt01_2B_full'

G = Greens(datadir, filename + ".h5", "G")
Sigma = Greens(datadir, filename + ".h5", "Sigma")
G.npy_output(datadir, filename + ".npy", "G")
Sigma.npy_output(datadir, filename + ".npy", "Sigma")


G_sep = Greens_sep(datadir, filename + ".h5", "G")
Sigma_sep = Greens_sep(datadir, filename + ".h5", "Sigma")
G_sep.npy_output(datadir, filename + ".npy", "G")
Sigma_sep.npy_output(datadir, filename + ".npy", "Sigma")

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
(15, 15, 1024, 129)
(15, 15, 1024, 1024)
(15, 15, 1024, 1024)


In [ ]:
datadir = '/pauli-storage/tblommel/NEdyson_data/He-VB2PP/equilibrium/'
filename = 'nt1000_k5_dt01_2B_full'

G_read = Greens(datadir, filename + ".npy", "G")
Sigma_read = Greens(datadir, filename + ".npy", "Sigma")

G_sep_read = Greens_sep(datadir, filename + ".npy", "G")
Sigma_sep_read = Greens_sep(datadir, filename + ".npy", "Sigma")

In [85]:
def compress_TV_block(G, datadir, GorSigma, filename):
    S = np.linalg.svd(G.GTV, full_matrices=False, compute_uv=False)
    np.save(datadir + "/block/TV/" + GorSigma + "_S_" + filename + '.npy', S)

def compress_TV_sep(G_sep, datadir, GorSigma, filename):
    nao = G_sep.GTV.shape[0]
    nt = G_sep.GTV.shape[2]
    ntau = G_sep.GTV.shape[3]
    
    S = np.zeros((nao, nao, ntau))
    
    for i in np.arange(nao):
        for j in np.arange(nao):
            S[i,j] = np.linalg.svd(G_sep.GTV[i,j], full_matrices=False, compute_uv = False)
    
    np.save(datadir + "/sep/TV/" + GorSigma + "_S_" + filename + '.npy', S)

In [86]:
datadir = '/pauli-storage/tblommel/NEdyson_data/He-VB2PP/compress/'
compress_TV_block(G, datadir, "G", filename = 'nt1000_k5_dt01_2B_full')
compress_TV_block(Sigma, datadir, "Sigma", filename = 'nt1000_k5_dt01_2B_full')
compress_TV_sep(G_sep, datadir, "G", filename = 'nt1000_k5_dt01_2B_full')
compress_TV_sep(Sigma_sep, datadir, "Sigma", filename = 'nt1000_k5_dt01_2B_full')

In [206]:
def compress_R_block(G, datadir, GorSigma, filename, levels):
    nao = G.nao
    nt = G.nt
        
    for l in levels:
        S = np.zeros((2**l, nao*nt//(2**(l+1))))
        size = (nt//(2**(l+1)))
        for block in np.arange(2**(l+1))+1:
            if block%2 == 1:
                S[(block-1)//2] = np.linalg.svd(G.GR[block*size*nao:(block+1)*size*nao,(block-1)*size*nao:block*size*nao], full_matrices=False, compute_uv=False)
        
        np.save(datadir + "/block/R/" + GorSigma + "_S_lv" + str(l) + "_" + filename + '.npy', S)
        
def compress_R_sep(G, datadir, GorSigma, filename, levels):
    nao = G.GR.shape[0]
    nt = G.GR.shape[2]
    for l in levels:
        S = np.zeros((nao, nao, 2**l, nt//(2**(l+1))))
        for i in np.arange(nao):
            for j in np.arange(nao):
                size = (nt//(2**(l+1)))
                for block in np.arange(2**(l+1))+1:
                    if block%2 == 1:
                        S[i,j,(block-1)//2] = np.linalg.svd(G.GR[i,j,block*size:(block+1)*size,(block-1)*size:block*size], full_matrices=False, compute_uv=False)
        
        np.save(datadir + "/sep/R/" + GorSigma + "_S_lv" + str(l) + "_" + filename + '.npy', S)
        
def compress_R_sep_diff_breakup(G, datadir, GorSigma, filename, levels):
    nao = G.GR.shape[0]
    nt = G.GR.shape[2]
    for l in levels:
        size = nt//(2**(l+2))
        S = np.zeros((nao, nao, 3*(2**(l+1)-1), size))
        S_excess = np.zeros((nao, nao, 2**(l+2)-1, size))
        for i in np.arange(nao):
            for j in np.arange(nao):
                #for d in np.arange(2**(l+1)-1)+1:
                #    S[i,j,3*(d-1)]   = np.linalg.svd(G.GR[i,j,d*2*size      : d*2*size+size      , (d-1)*2*size      : (d-1)*2*size+size     ], full_matrices=False, compute_uv=False)
                #    S[i,j,3*(d-1)+1] = np.linalg.svd(G.GR[i,j,d*2*size+size : d*2*size+size+size , (d-1)*2*size      : (d-1)*2*size+size     ], full_matrices=False, compute_uv=False)
                #    S[i,j,3*(d-1)+2] = np.linalg.svd(G.GR[i,j,d*2*size+size : d*2*size+size+size , (d-1)*2*size+size : (d-1)*2*size+size+size], full_matrices=False, compute_uv=False)
                for d in np.arange(2**(l+2)-1)+1:
                    S_excess[i,j,d-1]= np.linalg.svd(G.GR[i,j,d*size        : d*size+size        , (d-1)*size        : (d-1)*size+size       ], full_matrices=False, compute_uv=False)
   
        #np.save(datadir + "/sep/R/Diff_" + GorSigma + "_S_lv" + str(l) + "_" + filename + '.npy', S)
        np.save(datadir + "/sep/R/Diff_" + GorSigma + "_S_excess_lv" + str(l) + "_" + filename + '.npy', S_excess)


In [207]:
datadir = '/pauli-storage/tblommel/NEdyson_data/He-VB2PP/compress/'
#compress_R_block(G, datadir, "G", 'nt1000_k5_dt01_2B_full', np.arange(1,7))
#compress_R_block(Sigma, datadir, "Sigma", 'nt1000_k5_dt01_2B_full', np.arange(1,7))
#compress_R_sep(G_sep, datadir, "G", 'nt1000_k5_dt01_2B_full', np.arange(0,7))
#compress_R_sep(Sigma_sep, datadir, "Sigma", 'nt1000_k5_dt01_2B_full', np.arange(0,7))
compress_R_sep_diff_breakup(G_sep, datadir, "G", 'nt1000_k5_dt01_2B_full', np.arange(0,7))
compress_R_sep_diff_breakup(Sigma_sep, datadir, "Sigma", 'nt1000_k5_dt01_2B_full', np.arange(0,7))

In [208]:
def compress_L_block(G, datadir, GorSigma, filename, levels):
    nao = G.nao
    nt = G.nt
        
    for l in levels:
        S = np.zeros((2**l, nao*nt//(2**(l+1))))
        size = (nt//(2**(l+1)))
        for block in np.arange(2**(l+1))+1:
            if block%2 == 1:
                S[(block-1)//2] = np.linalg.svd(G.GL[block*size*nao:(block+1)*size*nao,(block-1)*size*nao:block*size*nao], full_matrices=False, compute_uv=False)
        
        np.save(datadir + "/block/L/" + GorSigma + "_S_lv" + str(l) + "_" + filename + '.npy', S)
        
def compress_L_sep(G, datadir, GorSigma, filename, levels):
    nao = G.GL.shape[0]
    nt = G.GL.shape[2]
    for l in levels:
        S = np.zeros((nao, nao, 2**l, nt//(2**(l+1))))
        for i in np.arange(nao):
            for j in np.arange(nao):
                size = (nt//(2**(l+1)))
                for block in np.arange(2**(l+1))+1:
                    if block%2 == 1:
                        S[i,j,(block-1)//2] = np.linalg.svd(G.GL[i,j,block*size:(block+1)*size,(block-1)*size:block*size], full_matrices=False, compute_uv=False)
        
        np.save(datadir + "/sep/L/" + GorSigma + "_S_lv" + str(l) + "_" + filename + '.npy', S)
        
def compress_L_sep_diff_breakup(G, datadir, GorSigma, filename, levels):
    nao = G.GL.shape[0]
    nt = G.GL.shape[2]
    for l in levels:
        size = nt//(2**(l+2))
        S = np.zeros((nao, nao, 3*(2**(l+1)-1), size))
        S_excess = np.zeros((nao, nao, 2**(l+2)-1, size))
        for i in np.arange(nao):
            for j in np.arange(nao):
                #for d in np.arange(2**(l+1)-1)+1:
                #    S[i,j,3*(d-1)]   = np.linalg.svd(G.GL[i,j,d*2*size      : d*2*size+size      , (d-1)*2*size      : (d-1)*2*size+size     ], full_matrices=False, compute_uv=False)
                #    S[i,j,3*(d-1)+1] = np.linalg.svd(G.GL[i,j,d*2*size+size : d*2*size+size+size , (d-1)*2*size      : (d-1)*2*size+size     ], full_matrices=False, compute_uv=False)
                #    S[i,j,3*(d-1)+2] = np.linalg.svd(G.GL[i,j,d*2*size+size : d*2*size+size+size , (d-1)*2*size+size : (d-1)*2*size+size+size], full_matrices=False, compute_uv=False)
                for d in np.arange(2**(l+2)-1)+1:
                    S_excess[i,j,d-1]= np.linalg.svd(G.GL[i,j,d*size        : d*size+size        , (d-1)*size        : (d-1)*size+size       ], full_matrices=False, compute_uv=False)
   
        #np.save(datadir + "/sep/L/Diff_" + GorSigma + "_S_lv" + str(l) + "_" + filename + '.npy', S)
        np.save(datadir + "/sep/L/Diff_" + GorSigma + "_S_excess_lv" + str(l) + "_" + filename + '.npy', S_excess)


In [209]:
datadir = '/pauli-storage/tblommel/NEdyson_data/He-VB2PP/compress/'
#compress_L_block(G, datadir, "G", 'nt1000_k5_dt01_2B_full', np.arange(0,7))
#compress_L_block(Sigma, datadir, "Sigma", 'nt1000_k5_dt01_2B_full', np.arange(0,7))
#compress_L_sep(G_sep, datadir, "G", 'nt1000_k5_dt01_2B_full', np.arange(0,7))
#compress_L_sep(Sigma_sep, datadir, "Sigma", 'nt1000_k5_dt01_2B_full', np.arange(0,7))
compress_L_sep_diff_breakup(G_sep, datadir, "G", 'nt1000_k5_dt01_2B_full', np.arange(0,7))
compress_L_sep_diff_breakup(Sigma_sep, datadir, "Sigma", 'nt1000_k5_dt01_2B_full', np.arange(0,7))

In [134]:
def TV_compress_amount_block(datadir, GorSigma, filename, eps, nt, nao):
    S = np.load(datadir + "/block/TV/" + GorSigma + "_S_" + filename + ".npy")
    ntau = S.shape[0]/nao
    for ep in eps:
        k = sum(S>ep)
        print(ep, k, k * (nt*nao + ntau*nao + 1) / (nt*ntau*nao*nao))

def TV_compress_amount_sep(datadir, GorSigma, filename, eps, nt, nao):
    S = np.load(datadir + "/sep/TV/" + GorSigma + "_S_" + filename + ".npy")
    nao = S.shape[0]
    ntau = S.shape[2]
    
    compress_amount = np.zeros(eps.shape[0])
    
    for i in np.arange(nao):
        for j in np.arange(nao):
            for e, ep in enumerate(eps):
                k = sum(S[i,j] > ep)
                compress_amount[e] += k * (nt + ntau + 1) / (nt*ntau*nao*nao)
    
    for e, ep in enumerate(eps):
        print(ep, compress_amount[e])


In [135]:
datadir = "/pauli-storage/tblommel/NEdyson_data/He-VB2PP/compress/"
eps = np.array([1e-6, 1e-8, 1e-10, 1e-12])
TV_compress_amount_block(datadir, "G", "nt1000_k5_dt01_2B_full", eps, 1024, 15)
TV_compress_amount_block(datadir, "Sigma", "nt1000_k5_dt01_2B_full", eps, 1024, 15)
TV_compress_amount_sep(datadir, "G", "nt1000_k5_dt01_2B_full", eps, 1024, 15)
TV_compress_amount_sep(datadir, "Sigma", "nt1000_k5_dt01_2B_full", eps, 1024, 15)

1e-06 198 0.11522286821705427
1e-08 283 0.16468723083548664
1e-10 360 0.20949612403100776
1e-12 448 0.2607062876830319
1e-06 260 0.1513027562446167
1e-08 350 0.20367678725236865
1e-10 440 0.2560508182601206
1e-12 531 0.3090067829457364
1e-06 0.020034722222222218
1e-08 0.027023578811369516
1e-10 0.03385712747631353
1e-12 0.04123425387596898
1e-06 0.024616305986218768
1e-08 0.03253701012058571
1e-10 0.04018592538759689
1e-12 0.04833959140826872


In [5]:
def R_compress_amount_block(datadir, GorSigma, filename, eps, nt, nao, numlevel):
    print(GorSigma + " compress fraction for block data")
    compress_amount = np.zeros(eps.shape[0])
    
    for l in np.arange(numlevel):
        S = np.load(datadir + '/block/R/' + GorSigma + "_S_" + "lv" + str(l) + "_" + filename + ".npy")
        
        if l==0:
            for e, ep in enumerate(eps):
                k = sum(S > ep)
                compress_amount[e] += k * (2 * nt * nao / 2**(l+1) + 1)
        else:
            for b in np.arange(2**l):
                for e, ep in enumerate(eps):
                    k = sum(S[b] > ep)
                    compress_amount[e] += k * (2 * nt * nao / 2**(l+1) + 1)
        
    full_lvl_dim = nt / 2**(numlevel)
    compress_amount += 2**numlevel * full_lvl_dim * (full_lvl_dim + 1) / 2 * nao * nao
    
    total_data = nt * (nt+1) / 2 * nao * nao
    
    for e, ep in enumerate(eps):
        print(ep, compress_amount[e]/total_data)

def R_compress_amount_sep(datadir, GorSigma, filename, eps, nt, nao, numlevel):
    print(GorSigma + " compress fraction for separated data")
    compress_amount = np.zeros(eps.shape[0])
    
    for l in np.arange(numlevel):
        S = np.load(datadir + '/sep/R/' + GorSigma + "_S_" + "lv" + str(l) + "_" + filename + ".npy")
        
        for i in np.arange(nao):
            for j in np.arange(nao):
                for b in np.arange(2**l):
                    for e, ep in enumerate(eps):
                        k = sum(S[i,j,b] > ep)
                        compress_amount[e] += k * (2 * nt / 2**(l+1) + 1)
        
    full_lvl_dim = nt / 2**(numlevel)
    compress_amount += 2**numlevel * full_lvl_dim * (full_lvl_dim + 1) / 2 * nao * nao
    
    total_data = nt * (nt+1) / 2 * nao * nao
    
    for e, ep in enumerate(eps):
        print(ep, compress_amount[e]/total_data)
        
def R_compress_amount_sep_diff(datadir, GorSigma, filename, eps, nt, nao, numlevel):
    print(GorSigma + " compress fraction for separated data different breakup")
    compress_amount = np.zeros(eps.shape[0])

    for l in np.arange(numlevel):
        S = np.load(datadir + "/sep/R/Diff_"+ GorSigma + "_S_lv" + str(l) + "_" + filename + '.npy')
        size = (nt//(2**(l+2)))

        for i in np.arange(nao):
            for j in np.arange(nao):
                for b in np.arange(S.shape[2]):
                    for e, ep in enumerate(eps):
                        k = sum(S[i,j,b] > ep)
                        compress_amount[e] += k * (2 * size + 1)
    
    if(numlevel > 0):
        S = np.load(datadir + "/sep/R/Diff_"+ GorSigma + "_S_excess_lv" + str(numlevel-1) + "_" + filename + '.npy')
        size = nt//(2**(numlevel+1))
        for i in np.arange(nao):
            for j in np.arange(nao):
                for b in np.arange(S.shape[2]):
                    for e, ep in enumerate(eps):
                        k = sum(S[i,j,b] > ep)
                        compress_amount[e] += k * (2 * size + 1)

    full_size = nt//(2**(numlevel+1))
    compress_amount += 2**(numlevel+1) * full_size * (full_size+1)/2 * nao * nao
    
    total_data = nt * (nt+1) / 2 * nao * nao
    
    for e, ep in enumerate(eps):
        print(ep, compress_amount[e]/total_data)

In [7]:
datadir = "/pauli-storage/tblommel/NEdyson_data/He-VB2PP/compress/"
eps = np.array([1e-6, 1e-8, 1e-10, 1e-12])
#R_compress_amount_block(datadir, "G", "nt1000_k5_dt01_2B_full", eps, 1024, 15, 7)
#R_compress_amount_block(datadir, "Sigma", "nt1000_k5_dt01_2B_full", eps, 1024, 15, 7)
#R_compress_amount_sep(datadir, "G", "nt1000_k5_dt01_2B_full", eps, 1024, 15, 7)
#R_compress_amount_sep(datadir, "Sigma", "nt1000_k5_dt01_2B_full", eps, 1024, 15, 7)
R_compress_amount_sep_diff(datadir, "G", "nt1000_k5_dt01_2B_full", eps, 1024, 15, 7)
R_compress_amount_sep_diff(datadir, "Sigma", "nt1000_k5_dt01_2B_full", eps, 1024, 15, 7)

G compress fraction for separated data different breakup
1e-06 0.03478920223577236
1e-08 0.04381557418699187
1e-10 0.05286404979674797
1e-12 0.06486328760162602
Sigma compress fraction for separated data different breakup
1e-06 0.044129700203252034
1e-08 0.05402070630081301
1e-10 0.06404141260162602
1e-12 0.07609166666666667


In [8]:
def L_compress_amount_block(datadir, GorSigma, filename, eps, nt, nao, numlevel):
    compress_amount = np.zeros(eps.shape[0])
    
    for l in np.arange(numlevel):
        S = np.load(datadir + '/block/L/' + GorSigma + "_S_" + "lv" + str(l) + "_" + filename + ".npy")
        for b in np.arange(2**l):
            for e, ep in enumerate(eps):
                k = sum(S[b] > ep)
                compress_amount[e] += k * (2 * nt * nao / 2**(l+1) + 1)
        
    full_lvl_dim = nt / 2**(numlevel)
    compress_amount += 2**numlevel * full_lvl_dim * (full_lvl_dim + 1) / 2 * nao * nao
    
    total_data = nt * (nt+1) / 2 * nao * nao
    
    for e, ep in enumerate(eps):
        print(ep, compress_amount[e]/total_data)

def L_compress_amount_sep(datadir, GorSigma, filename, eps, nt, nao, numlevel):
    compress_amount = np.zeros(eps.shape[0])
    
    for l in np.arange(numlevel):
        S = np.load(datadir + '/sep/L/' + GorSigma + "_S_" + "lv" + str(l) + "_" + filename + ".npy")
        
        for i in np.arange(nao):
            for j in np.arange(nao):
                for b in np.arange(2**l):
                    for e, ep in enumerate(eps):
                        k = sum(S[i,j,b] > ep)
                        compress_amount[e] += k * (2 * nt / 2**(l+1) + 1)
        
    full_lvl_dim = nt / 2**(numlevel)
    compress_amount += 2**numlevel * full_lvl_dim * (full_lvl_dim + 1) / 2 * nao * nao
    
    total_data = nt * (nt+1) / 2 * nao * nao
    
    for e, ep in enumerate(eps):
        print(ep, compress_amount[e]/total_data)
        
def L_compress_amount_sep_diff(datadir, GorSigma, filename, eps, nt, nao, numlevel):
    print(GorSigma + " compress fraction for separated data different breakup")
    compress_amount = np.zeros(eps.shape[0])

    for l in np.arange(numlevel):
        S = np.load(datadir + "/sep/L/Diff_"+ GorSigma + "_S_lv" + str(l) + "_" + filename + '.npy')
        size = (nt//(2**(l+2)))

        for i in np.arange(nao):
            for j in np.arange(nao):
                for b in np.arange(S.shape[2]):
                    for e, ep in enumerate(eps):
                        k = sum(S[i,j,b] > ep)
                        compress_amount[e] += k * (2 * size + 1)
    
    if(numlevel > 0):
        S = np.load(datadir + "/sep/L/Diff_"+ GorSigma + "_S_excess_lv" + str(numlevel-1) + "_" + filename + '.npy')
        size = nt//(2**(numlevel+1))
        for i in np.arange(nao):
            for j in np.arange(nao):
                for b in np.arange(S.shape[2]):
                    for e, ep in enumerate(eps):
                        k = sum(S[i,j,b] > ep)
                        compress_amount[e] += k * (2 * size + 1)

    full_size = nt//(2**(numlevel+1))
    compress_amount += 2**(numlevel+1) * full_size * (full_size+1)/2 * nao * nao
    
    total_data = nt * (nt+1) / 2 * nao * nao
    
    for e, ep in enumerate(eps):
        print(ep, compress_amount[e]/total_data)

In [9]:
datadir = "/pauli-storage/tblommel/NEdyson_data/He-VB2PP/compress/"
eps = np.array([1e-6, 1e-8, 1e-10, 1e-12])
#L_compress_amount_block(datadir, "G", "nt1000_k5_dt01_2B_full", eps, 1024, 15, 7)
#L_compress_amount_block(datadir, "Sigma", "nt1000_k5_dt01_2B_full", eps, 1024, 15, 7)
#L_compress_amount_sep(datadir, "G", "nt1000_k5_dt01_2B_full", eps, 1024, 15, 7)
#L_compress_amount_sep(datadir, "Sigma", "nt1000_k5_dt01_2B_full", eps, 1024, 15, 7)
L_compress_amount_sep_diff(datadir, "G", "nt1000_k5_dt01_2B_full", eps, 1024, 15, 7)
L_compress_amount_sep_diff(datadir, "Sigma", "nt1000_k5_dt01_2B_full", eps, 1024, 15, 7)

G compress fraction for separated data different breakup
1e-06 0.023500533536585367
1e-08 0.029742708333333333
1e-10 0.0368789210704607
1e-12 0.04354123475609756
Sigma compress fraction for separated data different breakup
1e-06 0.029260899390243903
1e-08 0.03648366361788618
1e-10 0.043905767276422765
1e-12 0.053169275067750675
